In [55]:
import os
import optuna
import shutil
from optuna.artifacts import FileSystemArtifactStore
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/")
import importlib
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *
#dtype = torch.float32
dtype = torch.double
torch.set_default_dtype(dtype)
torch.autograd.set_detect_anomaly(True)
# Set the device to CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import deepspeed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Using device: cuda


## Data load


In [61]:
#from src.trainer import num_samples

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/three_body_train_data.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.8)
#num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# Placeholder for input (magnitudes of velocities and accelerations)
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=dtype).to(device)
data = torch.empty((num_samples,6+num_features), dtype=dtype).to(device)
data[:,6:] = data_org


# Magnitudes vel, acc, mass
vel = torch.norm(data_org[:,4:7], p=2, dim=1)
acc = torch.empty((num_samples,4), dtype=dtype).to(device)
acc[:,0] = torch.norm(data_org[:,7:10], p=2, dim=1)
acc[:,1] = torch.norm(data_org[:,10:13], p=2, dim=1)
acc[:,2] = torch.norm(data_org[:,13:16], p=2, dim=1)
acc[:,3] = torch.norm(data_org[:,16:19], p=2, dim=1)
mass = data_org[:,0]

data[:,0] = vel
data[:,1:5] = acc
data[:,5] = mass

# Normalize the data
data_min = data[:,:6].min(axis=0, keepdim=True).values
data_max = data[:,:6].max(axis=0, keepdim=True).values
data[:,:6] = (data[:,:6] - data_min) / (data_max - data_min)


# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
# Define generator with a fixed seed
generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = random_split(dataset, [num_train, num_test], generator=generator)

# Create DataLoaders for the training and testing datasets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(data.shape)

torch.Size([15586, 68])


## Model load

In [85]:

input_size = 6     # Number of input features
input_mask = np.r_[0:6]
hidden_dims = [32,64,64,32] 
output_size = 2     # Number of output 
batch_size=64
TargetEnergyError =1e-2
# Load an existing study from your storage (e.g., SQLite)
study = optuna.load_study(
    study_name="three_body_preliminary",
    storage="sqlite:///../optuna/database/three_body_preliminary.db"
)

# Retrieve the trial (here we choose trial number 1, adjust as needed)
trial = next(t for t in study.trials if t.number == 576)
artifact_base_path = "../data/optuna/artifacts/"
file_path = artifact_base_path+f"/model_trial_{trial.number}.pt"

weights = {"time_step":0.1, "energy_loss":1e-1}

# Instantiate the model
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.1, output_positive=True).to(device)

# Load the state dictionary from a file
state_dict = torch.load(file_path, map_location=torch.device(device))
model.load_state_dict(state_dict)

# Set the model to evaluation mode (important for inference)
model.eval()

# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3DM(nParticle=3, nAttribute=20, nBatch=batch_size,alpha=0.1, beta=1.0, gamma=1.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)


test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
    validate(model, criterion, test_loader, input_mask, weights, device)
print(f"Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

cpath = artifact_base_path + f"cmodel_trial_{trial.number}.pt"
if os.path.exists(cpath):
    print("traced model exists.")
else:
    traced_model = torch.jit.script(model)
    traced_model.save(cpath)



Test Loss: 2.5099e+02, Energy Loss: 1.5071e+06/5.9096e-03, std: 3.5494e+07/1.9795e-01, Time step: 2.8580e-08/1.0262e-09


In [ ]:
18/287/49/50/412/337

In [ ]:
study.best_trials

[FrozenTrial(number=3, state=TrialState.COMPLETE, values=[6010.943933357334, 0.016313339423413512], datetime_start=datetime.datetime(2025, 3, 24, 21, 25, 37, 105252), datetime_complete=datetime.datetime(2025, 3, 24, 21, 28, 36, 866095), params={'optimizer': 'SGD', 'lr': 1.9142976961696882e-07}, user_attrs={'artifact_id': 'b709429e-57ac-41f9-950d-05c2744d7d42'}, system_attrs={'artifacts:b709429e-57ac-41f9-950d-05c2744d7d42': '{"artifact_id": "b709429e-57ac-41f9-950d-05c2744d7d42", "filename": "model_trial_3.pt", "mimetype": "application/octet-stream", "encoding": null}'}, intermediate_values={}, distributions={'optimizer': CategoricalDistribution(choices=('Adam', 'RMSprop', 'SGD')), 'lr': FloatDistribution(high=0.01, log=True, low=1e-09, step=None)}, trial_id=4, value=None),
 FrozenTrial(number=5, state=TrialState.COMPLETE, values=[6240.613105477894, 0.024051328635629952], datetime_start=datetime.datetime(2025, 3, 24, 21, 25, 47, 991052), datetime_complete=datetime.datetime(2025, 3, 24,

## Artifact

In [15]:
artifact_path = trial.artifacts.get("artifact_file", None)

AttributeError: 'FrozenTrial' object has no attribute 'artifacts'

In [23]:
artifact_base_path = "../data/optuna/artifacts/"
artifact_path = artifact_base_path+ trial.user_attrs['artifact_id']

In [40]:
# Load an existing study from your storage (e.g., SQLite)
study = optuna.load_study(
    study_name="three_body_preliminary",
    storage="sqlite:///../optuna/database/three_body_preliminary.db"
)

# Retrieve the trial (here we choose trial number 1, adjust as needed)
trial = next(t for t in study.trials if t.number == 1)

# Retrieve the artifact_id stored as a user attribute during the upload
artifact_id = trial.user_attrs["artifact_id"]
#artifact_base_path = "../data/optuna/artifacts/"
#file_path = artifact_base_path+f"/model_trial_{trial.number}.pt"
file_path = f"./downloaded_artifacts/model_trial_{trial.number}.txt"
artifact_store = FileSystemArtifactStore(base_path=artifact_base_path)


# Download the artifact without specifying an output directory
optuna.artifacts.download_artifact(
    artifact_id=artifact_id,
    file_path=file_path,
    artifact_store=artifact_store
    )
print(f"Artifact downloaded to: {file_path}")

# Optionally, if you want the file in a specific directory, move it manually:
#output_dir = "downloaded_artifacts"
#os.makedirs(output_dir, exist_ok=True)
#destination_path = os.path.join(output_dir, os.path.basename(artifact_path))
#shutil.move(artifact_path, destination_path)
#print(f"Artifact moved to: {destination_path}")

# If the artifact is a PyTorch model state dict, you can load it:
#state_dict = torch.load(destination_path)
#print("Loaded model state dict keys:", list(state_dict.keys()))

Artifact downloaded to: ./downloaded_artifacts/model_trial_1.txt
